In [1]:
import pybamm
import numpy as np 
import matplotlib.pyplot as plt
import pickle
from joblib import Parallel, delayed
from jupyterthemes import jtplot
jtplot.style(theme="monokai", context="notebook", ticks=True, grid=False)
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

from IPython.display import Audio
url = "https://proxy.notificationsounds.com/wake-up-tones/you-have-new-message-484/download/file-a1_ascendent-64kbps.mp3"
notify=Audio(url = url, autoplay = True)

########## pickle commands for dumping and retrieving data #######
# filename = 'dogs'
# outfile = open(filename,'wb')
# pickle.dump(dogs_dict,outfile)
# outfile.close()
# infile = open(filename,'rb')
# data = pickle.load(infile)
# infile.close()

########### parallel computations command ####################
# data = Parallel(n_jobs=-1)(delayed(my_function)(i) for i in my_iterable)

In [15]:
model = pybamm.lithium_sulfur.ZhangEtAl2015()
params = model.default_parameter_values

R = model.param.R.value
F = model.param.F.value
T = params[model.param.T_ref.name]
N = model.param.N.value

# model-specific known parameters
'''
For the stochiometric sij paramters, i indicates species of Sulfur and j reaction number.
For example, S6 in reaction 3 has s63.
The one exception is for S8(s), which has S instead of an integer. 
'''
sLi1 = params[model.param.sLi1.name]
sS2 = params[model.param.sS2.name]
s82 = params[model.param.s82.name]
s83 = params[model.param.s83.name]
s63 = params[model.param.s63.name]
s64 = params[model.param.s64.name]
s44 = params[model.param.s44.name]
s45 = params[model.param.s45.name]
s25 = params[model.param.s25.name]
s26 = params[model.param.s26.name]
s16 = params[model.param.s16.name]

E01 = params[model.param.E01.name]
E02 = params[model.param.E02.name]
E03 = params[model.param.E03.name]
E04 = params[model.param.E04.name]
E05 = params[model.param.E05.name]
E06 = params[model.param.E06.name]

n1 = -params[model.param.n1.name]*-1
n2 = -params[model.param.n2.name]*-1
n3 = -params[model.param.n3.name]*-1
n4 = -params[model.param.n4.name]*-1
n5 = -params[model.param.n5.name]*-1
n6 = -params[model.param.n6.name]*-1

i02 = params[model.param.i02.name]
i03 = params[model.param.i03.name]
i04 = params[model.param.i04.name]
i05 = params[model.param.i05.name]
i06 = params[model.param.i06.name]

A = params[model.param.Area.name]
l = params[model.param.l.name]
V_Li2S = params[model.param.V_Li2S.name]
nu0_Li2S = params[model.param.nu0_Li2S.name]
Li0 = params[model.param.Li0.name]
k_p = params[model.param.k_p.name]
K_sp = params[model.param.K_sp.name]
av0 = params[model.param.av0.name]
ep0 = params[model.param.ep0.name]
sigma0 = params[model.param.sigma0.name]
xi = params[model.param.xi.name]
b = params[model.param.b.name]
eta1 = params[model.param.eta1.name]


In [16]:
model1 = pybamm.BaseModel()

In [17]:
# set internal variables
S8s = pybamm.Variable("S8(s) [g]")
S8 = pybamm.Variable("S8 [g]")
S6 = pybamm.Variable("S6 [g]")
S4 = pybamm.Variable("S4 [g]")
S2 = pybamm.Variable("S2 [g]")
S = pybamm.Variable("S [g]")
eta2 = pybamm.Variable("Reaction 2 overpotential [V]") 

In [18]:
# Lithium ion concentration as defined in equation (10) in [1]
Li = 2*( S8 + S6 + S4 + S2 + S) + Li0  

# Reduction Potential as given in Equation (14) of [1]
E1 = E01 - (R*T/(n1*F))*sLi1*pybamm.log(Li/1000) 

# Reduction Potential as given in Equation (14) of [1]
E2 = E02 - (R*T/(n2*F))*pybamm.log(((S8s/1000)**sS2)*((S8/1000)**s82 ))

# Reduction Potential as given in Equation (14) of [1]
E3 = E03 - (R*T/(n3*F))*pybamm.log(((S8/1000)**s83)*((S6/1000)**s63))

# Reduction Potential as given in Equation (14) of [1]
E4 = E04 - (R*T/(n4*F))*pybamm.log(((S6/1000)**s64)*((S4/1000)**s44))

# Reduction Potential as given in Equation (14) of [1]
E5 = E05 - (R*T/(n5*F))*pybamm.log(((S4/1000)**s45)*((S2/1000)**s25))

# Reduction Potential as given in Equation (14) of [1]
E6 = E06 - (R*T/(n6*F))*pybamm.log(((S2/1000)**s26)*((S/1000)**s16)) 

# Overpotential implied by equation (8) in [1]
eta3 = E2 + eta2 - E3

# Overpotential implied by equation (8) in [1]
eta4 = E2 + eta2 - E4

# Overpotential implied by equation (8) in [1]
eta5 = E2 + eta2 - E5

# Overpotential implied by equation (8) in [1]
eta6 = E2 + eta2 - E6

# Reaction current functions defined in equation (12) in [1]
i2 = 2*i02*pybamm.sinh(n2*F*eta2/(2*R*T))

# Reaction current functions defined in equation (12) in [1]
i3 = 2*i03*pybamm.sinh(n3*F*eta3/(2*R*T))

# Reaction current functions defined in equation (12) in [1]
i4 = 2*i04*pybamm.sinh(n4*F*eta4/(2*R*T))

# Reaction current functions defined in equation (12) in [1]
i5 = 2*i05*pybamm.sinh(n5*F*eta5/(2*R*T))

# Reaction current functions defined in equation (12) in [1]
i6 = 2*i06*pybamm.sinh(n6*F*eta6/(2*R*T))

# Precipitation rate defined in equation (11) in [1]
r_p = k_p*nu0_Li2S*((Li**2)*S - K_sp)

# Electrolyte conductivity as defined in equation (15) in [1]
sigma =  (sigma0 - b*pybamm.AbsoluteValue(Li-Li0))

# Electrolyte Resistance is given in [1] on page 3
Rs = l/(A*sigma)

# Voltage as defined by equation (8) in [1]
I = .34/5
V = E2 + eta2 + (E1 + eta1) - I*Rs

In [19]:
# Differential equation (9) in [1]
dS8sdt = (av0/F)*(sS2*i2/n2) + S8s*V_Li2S*r_p

# Differential equation (9) in [1]
dS8dt = (av0/F)*( (s82*i2/n2) + (s83*i3/n3) ) + S8*V_Li2S*r_p

# Differential equation (9) in [1]
dS6dt = (av0/F)*( (s63*i3/n3) + (s64*i4/n4) ) + S6*V_Li2S*r_p

# Differential equation (9) in [1]
dS4dt = (av0/F)*( (s44*i4/n4) + (s45*i5/n5) ) + S4*V_Li2S*r_p

# Differential equation (9) in [1]
dS2dt = (av0/F)*( (s25*i5/n5) + (s26*i6/n6) ) + S2*V_Li2S*r_p

# Differential equation (9) in [1]
dSdt = (av0*i6/F) - r_p + S*V_Li2S*r_p 

In [20]:
algebraic_condition =  av0*(i2 + i3 + i4 + i5 + i6) - (I/(A*l))

In [21]:
model1.algebraic.update({eta2: algebraic_condition})
model1.rhs.update({
                         S8s: dS8sdt,
                         S8: dS8dt,
                         S6 : dS6dt,
                         S4: dS4dt, 
                         S2: dS2dt, 
                         S: dSdt
                        })


In [22]:
model1.initial_conditions.update(
            {
                S8s: params[model.param.zhang_S8s_initial.name],
                S8: params[model.param.zhang_S8_initial.name],
                S6: params[model.param.zhang_S6_initial.name],
                S4 : params[model.param.zhang_S4_initial.name],
                S2 : params[model.param.zhang_S2_initial.name],
                S : params[model.param.zhang_S_initial.name],
                eta2 : params[model.param.zhang_eta2_initial.name]          
            }
        )

In [23]:
model1.variables.update(
            {
                "Time [s]": pybamm.t * model1.timescale,
                "S8(s) [g]": S8s,
                "S8 [g]": S8,
                "S6 [g]": S6,
                "S4 [g]": S4,
                "S2 [g]": S2,
                "S [g]": S,
                "Li [g]" : Li,
                "Terminal voltage [V]" : V,
                "Electrolyte Resistance [S-1]" : Rs,
                "Conductivity [m.S-1]" : sigma,
                "Precipitation rate [s-1]": r_p,
                "Reaction 1 potential [V]": E1,
                "Reaction 2 potential [V]": E2,
                "Reaction 3 potential [V]": E3,
                "Reaction 4 potential [V]": E4,
                "Reaction 5 potential [V]": E5,
                "Reaction 6 potential [V]": E6,
                "Reaction 1 overpotential [V]": eta1,
                "Reaction 2 overpotential [V]": eta2,
                "Reaction 3 overpotential [V]": eta3,
                "Reaction 4 overpotential [V]": eta4,
                "Reaction 5 overpotential [V]": eta5,
                "Reaction 6 overpotential [V]": eta6,
                "Reaction 2 current [A]": i2,
                "Reaction 3 current [A]": i3,
                "Reaction 4 current [A]": i4,
                "Reaction 5 current [A]": i5,
                "Reaction 6 current [A]": i6,
                "Algebraic condition": algebraic_condition,
                "dS8sdt": dS8sdt,
                "dS8dt": dS8dt,
                "dS6dt": dS6dt,
                "dS4dt": dS4dt, 
                "dS2dt": dS2dt, 
                "dSdt" : dSdt
            }
        )

In [24]:
solver = pybamm.CasadiSolver()
t = np.linspace(0, 1, 20)
solution = solver.solve(model1, t)

AttributeError: 'float' object has no attribute 'id'

In [25]:
sim0 = pybamm.Simulation(
    model1,
    solver=pybamm.CasadiSolver(
        atol=1e-12, 
        rtol=1e-6, 
        extra_options_setup={"max_step_size": 0.1}, 
        max_step_decrease_count = 15,
        root_method = 'lm',
        root_tol = 1e-7, 
        dt_max = 1e-7
    ),
)
sim0.solve(np.linspace(0, 1, 1000))

AttributeError: 'float' object has no attribute 'evaluates_to_number'

In [26]:
model1.variables

{'Time [s]': Multiplication(0x7873608854d2ffb4, *, children=['time', '1.0'], domain=[], auxiliary_domains={}),
 'S8(s) [g]': Variable(-0x5078a7f0c2d80edf, S8(s) [g], children=[], domain=[], auxiliary_domains={}),
 'S8 [g]': Variable(-0x9e6deabfb3ec56, S8 [g], children=[], domain=[], auxiliary_domains={}),
 'S6 [g]': Variable(0x38da8d6ec86112b6, S6 [g], children=[], domain=[], auxiliary_domains={}),
 'S4 [g]': Variable(-0x35ada4cceb26e22b, S4 [g], children=[], domain=[], auxiliary_domains={}),
 'S2 [g]': Variable(-0x64780ca4b6fe875b, S2 [g], children=[], domain=[], auxiliary_domains={}),
 'S [g]': Variable(-0x1cc3d1274031fda7, S [g], children=[], domain=[], auxiliary_domains={}),
 'Li [g]': Addition(0x5789fdf9f47c33e1, +, children=['2.0 * (S8 [g] + S6 [g] + S4 [g] + S2 [g] + S [g])', '1100.0'], domain=[], auxiliary_domains={}),
 'Terminal voltage [V]': Subtraction(-0x32cc8339bf51c133, -, children=['2.38 - (0.025679653121600303 * log(((S8(s) [g] / 1000.0) ** -0.5) * ((S8 [g] / 1000.0) **